In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

### Data Reading


In [0]:
df = spark.read.format("parquet").load("abfss://bronze@sssadatabricksete.dfs.core.windows.net/orders")

In [0]:
display(df)

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("order_date",to_timestamp(col('order_date')))

In [0]:
display(df)

In [0]:
df =df.withColumn("year",year(col("order_date")))

In [0]:
df.display()

In [0]:
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

### Classes - OOP

In [0]:
class Windows:
  def dense_rank(self,df):
    df_dense_rank = df.withColumn("dense_rank",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_dense_rank
  
  def rank(self,df):
    df_rank = df.withColumn("rank",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_rank
  
  def row_number(self,df):
    df_row_number = df.withColumn("row_number",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_row_number

In [0]:
df_new =df

In [0]:
df_new.display()

In [0]:
obj = Windows()


In [0]:
df_result = obj.dense_rank(df_new)

In [0]:
display(df_result)

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@sssadatabricksete.dfs.core.windows.net/orders")

In [0]:
%sql
create table if not exists databricks_cata.silver.orders_silver
using delta
location "abfss://silver@sssadatabricksete.dfs.core.windows.net/orders"